<a href="https://colab.research.google.com/github/jepilogo97/nlp/blob/main/nlp-with-bert/nlp_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP con GPT

##### Jean Pierre Londoño González
##### Mini-Proyecto de clasificación de texto usando GPT
##### 21SEP2025

En este notebook harémos uso de un modelo GPT neo de 2.7B que utilizaremos para generar texto a partir de un contexto inicial que proveerémos. Luego, harémos fine tuning a este modelo con un dataset de podcast en inglés del investigador de IA Lex Fridman y observar como cambia la generación de texto en función del dataset que utilicemos.

#### Referencias
- Dataset: https://huggingface.co/datasets/RamAnanth1/lex-fridman-podcasts
- https://huggingface.co/EleutherAI/gpt-neo-2.7B

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la manipulación de datos y la construcción del modelo. Esto incluye NumPy y pandas para el manejo y análisis de datos; Hugging Face Datasets y Transformers para la carga de corpus y la tokenización; y PyTorch junto con PyTorch Lightning para definir, entrenar y evaluar el modelo de manera estructurada.

In [2]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

C:\Users\jepil\AppData\Local\Temp\ipykernel_16548\2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [8]:
!wget -O requirements.txt https://raw.githubusercontent.com/jepilogo97/nlp/main/nlp-with-gpt/requirements.txt
!pip install -r requirements.txt

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
"test" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"test" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"test" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"test" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"test" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [17]:
# Procesamiento de lenguaje natural y utilidades
import numpy as np  # Cálculo numérico y manejo de arreglos multidimensionales
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame

pd.set_option("display.max_rows", None)     # Todas las filas
pd.set_option("display.max_columns", None)  # Todas las columnas
pd.set_option("display.width", None)        # No cortar líneas

from datasets import Dataset, load_dataset, concatenate_datasets  # Carga y combinación de datasets de Hugging Face
from datasets.dataset_dict import DatasetDict
from collections import Counter  # Conteo de frecuencias de elementos (tokens, palabras, etc.)
import os  # Manejo de rutas, archivos y operaciones del sistema de archivos
import math  # Funciones matemáticas avanzadas (logaritmos, potencias, trigonometría, etc.)

import matplotlib.pyplot as plt
# Deep Learning con PyTorch
import torch  # Librería principal de tensores y operaciones en GPU/CPU
import torch.nn as nn  # Definición de capas y módulos de redes neuronales
import torch.nn.functional as F  # Funciones de activación y operaciones matemáticas de redes
from torch.utils.data import random_split, DataLoader, Subset  # Utilidades para crear y dividir datasets, cargar lotes y trabajar con subconjuntos
from torchinfo import summary

# Entrenamiento estructurado con PyTorch Lightning
from pytorch_lightning import LightningModule, Trainer  # Clase base y manejador de entrenamiento de modelos
from pytorch_lightning.loggers import TensorBoardLogger  # Registro de métricas e historial en TensorBoard
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint  # Detener entrenamiento si no mejora la métrica
from torchmetrics import Accuracy  # Métrica de precisión para clasificación supervisada

# Tipado para mayor legibilidad y validación de funciones
from typing import Optional, Tuple # Definición de tipos de datos para funciones y estructuras

from tqdm.auto import tqdm  # Barra de progreso adaptable para bucles
from transformers import AutoTokenizer, AutoModelForCausalLM  # Tokenizador automático de modelos preentrenados de Hugging Face
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode  # Conversión de bytes a caracteres Unicode (usado en tokenización tipo GPT-2)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

# Métricas de evaluación con Scikit-learn
from sklearn.model_selection import train_test_split  # División de datos en conjuntos de entrenamiento y prueba
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas de evaluación de modelos de clasificación

### 2. Generative pre-training Transformer - GPT

Los modelos tipo GPT, introducidos por Radfor, et.al., de OpenAI, al igual que los modelos BERT, hacen uso extensivo de la arquitectura de transformers como hemos estado viendo. Las diferencias claves se podrían resumir en:

1. GPT utiliza bloques de **Transformer Decoder** encadenados, mientras que el modelo BERT utiliza bloques de *Transformer Encoder*
2. GPT se centra en la generación de texto basado en un contexto, la tarea principal es la predicción del siguiente token en la secuencia, mientras que BERT se centra en el completado de partes de una secuencia, en función de un contexto anterior y posterior a la secuencia de entrada. Entonces BERT se centra en la construicción de representación de lenguage, mientras que GPT se centra en la generación de texto en función de un contexto.

Sin embargo, ambos se basan en la misma premisa de pre-entrenar el modelo en tareas no-supervisadas o semi-supervisadas para que el modelo aprenda las representaciones semánticas del lenguage y luego al modelo se le pueda hacer fine tuning a tareas posteriores.

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "EleutherAI/gpt-neo-2.7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2560)
    (wpe): Embedding(2048, 2560)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x GPTNeoBlock(
        (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
        )
        (ln_2): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2560, out_features=10240, bias=True)
          (c_proj)

In [11]:
modules = [m for m, _ in model.named_modules()]
modules

['',
 'transformer',
 'transformer.wte',
 'transformer.wpe',
 'transformer.drop',
 'transformer.h',
 'transformer.h.0',
 'transformer.h.0.ln_1',
 'transformer.h.0.attn',
 'transformer.h.0.attn.attention',
 'transformer.h.0.attn.attention.attn_dropout',
 'transformer.h.0.attn.attention.resid_dropout',
 'transformer.h.0.attn.attention.k_proj',
 'transformer.h.0.attn.attention.v_proj',
 'transformer.h.0.attn.attention.q_proj',
 'transformer.h.0.attn.attention.out_proj',
 'transformer.h.0.ln_2',
 'transformer.h.0.mlp',
 'transformer.h.0.mlp.c_fc',
 'transformer.h.0.mlp.c_proj',
 'transformer.h.0.mlp.act',
 'transformer.h.0.mlp.dropout',
 'transformer.h.1',
 'transformer.h.1.ln_1',
 'transformer.h.1.attn',
 'transformer.h.1.attn.attention',
 'transformer.h.1.attn.attention.attn_dropout',
 'transformer.h.1.attn.attention.resid_dropout',
 'transformer.h.1.attn.attention.k_proj',
 'transformer.h.1.attn.attention.v_proj',
 'transformer.h.1.attn.attention.q_proj',
 'transformer.h.1.attn.attentio

Observermos un ejemplo de generación simple.

In [13]:
text = "As part of MIT course 6S099, Artificial General Intelligence,"
best = 10

with torch.no_grad():
    tokens = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    output = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", output.shape)
    probs = torch.softmax(output, dim=-1)
    print("Dimensiones de la probabilidad de los tokens:", probs.shape)
    sorted_probs = torch.argsort(probs, dim=-1, descending=True)
    print({tokenizer.decode(token): f"{prob.cpu().numpy() * 100:.2f}%" for token, prob in zip(sorted_probs[:best], probs[sorted_probs[:best]])})

Dimensiones de la entrada: torch.Size([1, 14])
Dimensiones de la salida: torch.Size([1, 14, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de la probabilidad de los tokens: torch.Size([50257])
{' we': '26.56%', ' students': '15.52%', ' I': '7.04%', ' the': '5.74%', ' a': '2.97%', ' you': '2.32%', ' Professor': '1.70%', ' our': '1.43%', ' this': '1.11%', ' researchers': '1.07%'}


### 2. Implementando una función de generación

Ahora, la idea es que este modelo nos sirva para generar texto de forma recurrente e incremental. En la última capa de los modelos tipo GPT encontrarémos un tensor con forma $(b, s, v)$, donde:

- $b$: Es el tamaño del bache, o la cantidad de secuencias a procesar.
- $s$: Es la longitud de la secuencia de entrada.
- $v$: Es el tamaño del vocabulario del modelo, cuantos tokens soporta.

Pero este es el tensor de salida, por qué tiene la forma de la secuencia de entrada?, porque cada posición en la salida corresponde a la la predicción del siguiente token de esa posición en la secuencia de entrada. En otras palabras, lo que obtenemos como predicción, es una secuencia de igual tamaño a la de entrada, movida un token hacia adelante, lo que efectivamente nos predice un solo token a la vez y por ende, el token que nos insteresa, es el último.

Lo que obtenemos en este tensor es además los logits de TODO el vocabulario del modelo, con los cuales podemos calcular las probabilidades de que cada uno sea el que continue en la secuencia. Hay varias formas de decodificar el siguiente token, la más fácil de implementar sería una decodificación codiciosa (greedy) del siguiente token, que consiste simplemente en seleccionar el token con la probabilidad más alta. Este es un enfoque simple y efectivo para algunos casos, pero al mismo tiempo sufre de poca variabilidad e incluso puede caer en generación repetitiva.

Otra opción es el muestreo, ya que justamente podemos obtener probabilidades del siguiente token, lo más lógico sería muestrear con esas opciones de probabilidad, de este modo podemos obtener mayor diversidad a la hora de generar el texto, al costo eso si de que haya mayor aleatoridad ya que se le daría la oportunidad a incluso tokens con baja probabilidad, de ser seleccionados.

Otra opción podría ser un balanceo entre una decodificación greedy y una por muestreo, en función de otro hiperparámetro que podemos definir. Esta sería una técnica muy común en el contexto de Reinforcement Learning llamade e-greedy. Se hace la aclaración de que en este ejemplo no harémos nada de RL, solamente se hace mención de esta técnica para balancear entre explotación y exploración.

In [19]:
def generate(
        model: nn.Module, 
        tokenizer: PreTrainedTokenizerBase, 
        start: str, 
        max_length: int = 1000, 
        eps: float = 0.5, 
        top_n: int = 5,
        return_iterations: bool = False,
        device: str = "cpu") -> Tuple[str, Optional[pd.DataFrame]]:

    output = [start]
    iterations = []
    with torch.no_grad():
        input_ids = tokenizer(output[-1], return_tensors='pt')['input_ids'].to(device)
        for _ in range(max_length):
            # Tomamos los logits producidos por la última capa del modelo
            # Estos corresponden al siguiente token por cada posición de la cadena
            logits = model(input_ids=input_ids).logits
            # Por lo tanto, el que nos interesa es el último, que correspondería a la
            # predicción del siguiente token después del final de la cadena original
            # A este aplicamos un softmax para obtener las probabilidades por cada
            # token del vocabulario para estar presente en la cadena.
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Simplemente ordenamos por probabilidad de forma descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Utilizamos una politica tipo e-greedy para obtener el siguiente token de la secuencia
            # Un eps>=1 quiere decir que siempre se va seleccionar el token de forma 'greedy', es decir
            # siempre se toma el token con probabilidad más alta.

            # Un eps=0 quiere decir que siempre se va a muestrear el siguiente token en función
            # de las probabilidades de cada token

            # Un 0<eps<1 va a balancear de forma binomial entre tomar el token con la
            # probabilidad más alta y muestrear el token en función de sus probabilidades. 
            if np.random.random_sample(1)[0] < eps:
                # Se toma el mejor token
                next_token = sorted_tokens[0].unsqueeze(dim=0)
            else:
                # Se muetrea el token de la probabilidad de distribución
                next_token = torch.multinomial(probs, 1)
            
            if return_iterations:
                # Mantenemos pista de todas las iteraciones para análisis
                iteration = {'input': ''.join(output)}
                best_n = sorted_tokens[:top_n].cpu().tolist()
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})' for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            output.append(tokenizer.decode(next_token))
            input_ids = torch.cat([input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        output_text = ''.join(output)
        if not return_iterations:
            return output_text, None
        else:
            df = pd.DataFrame(iterations)
            return output_text, df

Ahora observemos que pasa cuando generamos texto con nuestra función y algunos parámetros.

Primero, observemos que pasa cuando pasamos un `eps=1` que quiere decir que la generación va a ser de tipo greedy:

In [20]:
output_text, iterations_df = generate(model, tokenizer, text, max_length=15, eps=1.0, top_n=10, return_iterations=True, device=device)
print(output_text)
iterations_df.head(15)

As part of MIT course 6S099, Artificial General Intelligence, we are going to explore the concept of ��artificial general intelligence�


,input,Choice #1,Choice #2,Choice #3,Choice #4,Choice #5,Choice #6,Choice #7,Choice #8,Choice #9,Choice #10
0,"As part of MIT course 6S099, Artificial Genera...",we (0.2656),students (0.1552),I (0.0704),the (0.0574),a (0.0297),you (0.0232),Professor (0.0170),our (0.0143),this (0.0111),researchers (0.0107)
1,"As part of MIT course 6S099, Artificial Genera...",are (0.1241),will (0.0959),� (0.0899),have (0.0875),were (0.0195),present (0.0182),use (0.0174),designed (0.0150),asked (0.0141),'ve (0.0133)
2,"As part of MIT course 6S099, Artificial Genera...",going (0.1412),studying (0.0660),exploring (0.0427),asked (0.0319),investigating (0.0308),building (0.0307),now (0.0305),working (0.0295),interested (0.0292),using (0.0266)
3,"As part of MIT course 6S099, Artificial Genera...",to (0.9735),through (0.0095),over (0.0038),\n (0.0021),on (0.0013),into (0.0010),back (0.0009),deep (0.0006),explore (0.0005),for (0.0003)
4,"As part of MIT course 6S099, Artificial Genera...",explore (0.1124),be (0.0773),look (0.0617),learn (0.0598),build (0.0586),study (0.0550),use (0.0515),discuss (0.0491),talk (0.0254),take (0.0220)
5,"As part of MIT course 6S099, Artificial Genera...",the (0.3220),how (0.0918),a (0.0813),some (0.0704),what (0.0414),an (0.0186),one (0.0158),machine (0.0146),two (0.0126),artificial (0.0114)
6,"As part of MIT course 6S099, Artificial Genera...",concept (0.0940),idea (0.0676),question (0.0344),topic (0.0340),field (0.0289),notion (0.0193),following (0.0181),relationship (0.0177),problem (0.0170),concepts (0.0156)
7,"As part of MIT course 6S099, Artificial Genera...",of (0.9646),and (0.0152),that (0.0073),", (0.0013)",behind (0.0008),called (0.0008),artificial (0.0007),Artificial (0.0007),in (0.0007),known (0.0006)
8,"As part of MIT course 6S099, Artificial Genera...",� (0.0651),a (0.0564),AI (0.0433),artificial (0.0381),Artificial (0.0349),general (0.0324),self (0.0263),intelligence (0.0245),an (0.0223),the (0.0208)
9,"As part of MIT course 6S099, Artificial Genera...",� (0.8114),� (0.1840),� (0.0019),� (0.0012),� (0.0010),� (0.0001),� (0.0001),� (0.0000),� (0.0000),� (0.0000)


Observamos como el input progresa a la vez que las opciones de tokens que hay. Sin importar cuantas veces invoquemos a la función con los mismos parámetros, siempre vamos a obtener los mismos resultados.

Ahora, observemos que pasa si introducimos exploración al reducir el `eps=0.5`, lo cual nos dice que aproximadamente la mitad de las veces va a elegir el siguiente token muestreando y la otra mitad explotando.

In [22]:
output_text, _ = generate(model, tokenizer, text, max_length=1000, eps=0.5, device=device)
print(output_text)

KeyboardInterrupt: 

### 3. Generando texto con las utilidades del modelo

Ahora, la clase de Huggingface implementa la función `generate` que hace la labor de generación por nosotros, incluyendo las opciones de muestreo y explotación como hemos observado. Solo que además permite otra serie de parámetros y opciones para controlar la generación de texto.

In [ ]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=1000, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output[0]))

### 4. Carga de dataset

Ahora, intentemos hacer fine tuning a nuestro modelo:

In [32]:
dataset = load_dataset("RamAnanth1/lex-fridman-podcasts")
dataset

In [ ]:
dataset['train'][0]

In [ ]:
dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df.head(10)

In [ ]:
df['Palabras por podcast'] = df['captions'].str.split().apply(len)
df['Palabras por podcast'].median()

### 5. Fine tuning

Aquí podemos observar que la mediana de longitud en terminos de palabras es de 31. Esto es esperado, pues los chistes deben ser cortos por naturaleza. Por otra parte, es bastante claro que el corpus original del modelo pre-entrenado contenía texto muy diferente a este, por lo que la calidad de los resultados, sin hacer mayores modificaciones puede que no sea buena.

Ahora, prepararémos el conjunto de datos para entrenamiento.

In [ ]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(examples['text'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function

Los modelos GPT no esperan otra cosa más que los `input_ids`, por lo que retirarémos todas las demás columnas del dataset ya que no nos son de utilidad en este momento. 

In [ ]:
dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=512), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')
tokenized_dataset

Finalmente procedemos a definir el entrenamiento.

In [ ]:
batch_size = 8 if IN_COLAB else 2
logging_steps = len(tokenized_dataset['train']) // batch_size
# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps,
    report_to='none'
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

In [ ]:
%%time
trainer.train()

### 6. Resultados

Ahora observemos los resultados.

In [ ]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=1000, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

No parece ser muy gracioso precisamente, sin embargo, notemos que la generación de texto cambia de "estilo", ahora es mucho más frecuente encontrar conversaciones cortas, frases concisas, y situaciones particulares, en lugar del estilo más literario del modelo original. Esto es un indicio de la influencia que tiene el conjunto de datos de entrenamiento en el modelo final, esto es algo a tener muy en cuenta a la hora de utilizar y hacer fine tuning a modelos de lenguaje.

### 7. Conclusiones

#### Eficacia del flujo de análisis

- Al comparar arquitecturas de modelos de lenguaje como BERT y GPT, se identifican similitudes en su capacidad de generar representaciones ricas del lenguaje, algunas diferencias clave en su estructura y en su proceso de entrenamiento.

- Ambos modelos demuestran que un pre-entrenamiento sólido y la construcción de embeddings de alta calidad son factores críticos para alcanzar buenos resultados en tareas posteriores, evitando costos de entrenamiento desde cero.

#### Rendimiento del modelo

- Tanto BERT como GPT pueden adaptarse a una amplia variedad de tareas posteriores (clasificación, generación de texto, análisis semántico, etc.), lo que valida la versatilidad de los enfoques de transfer learning y fine tuning.

- La elección del modelo y la estrategia de entrenamiento depende de la tarea: BERT sobresale en comprensión y análisis de contexto bidireccional, mientras que GPT destaca en generación de texto coherente y fluido.

#### Limitaciones observadas

- Los modelos generativos enfrentan un dilema de exploración–explotación: una decodificación enfocada en la explotación (p. ej. greedy search) brinda mayor precisión pero tiende a producir textos monótonos; en cambio, la exploración (p. ej. sampling con temperatura alta) promueve creatividad y diversidad, pero con riesgo de incoherencia o “alucinaciones”.

- La calidad del modelo depende en gran medida de los datos de entrenamiento. Conjuntos de datos sesgados, poco representativos o de baja calidad pueden degradar el desempeño e introducir sesgos o errores difíciles de corregir.

#### Áreas de mejora

- Profundizar en la selección y curaduría de datasets, asegurando diversidad, equilibrio y relevancia para el dominio de aplicación.

- Experimentar con estrategias de decodificación y con la afinación de hiperparámetros para encontrar el punto óptimo entre creatividad, coherencia y precisión.

- Explorar técnicas de optimización y compresión que permitan desplegar modelos grandes en entornos de recursos limitados.

#### Valor práctico

- La adopción de modelos pre-entrenados como GPT ofrece una base sólida y flexible para proyectos de NLP, equilibrando costo, tiempo y calidad.

- El entendimiento de los trade-offs entre exploración y explotación, así como la adecuada selección de datos y métodos de decodificación, es esencial para alinear el modelo con los objetivos específicos del negocio y minimizar riesgos de sesgos o resultados no deseados.

### 8. Apendice

In [ ]:
import pkg_resources

libs = [
    "numpy",
    "pandas",
    "datasets",
    "torch",
    "pytorch-lightning",
    "torchmetrics",
    "tqdm",
    "transformers",
    "scikit-learn"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except Exception:
        print(f"{lib}")

In [ ]:
 ## Solo correr en local

# import nbformat

## Cargar notebook
# with open("nlp_with_bert.ipynb", "r", encoding="utf-8") as f:
    # nb = nbformat.read(f, as_version=4)

## Eliminar widgets corruptos si existen
# if "widgets" in nb["metadata"]:
    # del nb["metadata"]["widgets"]

## Guardar reparado
# with open("nlp_with_bert.ipynb", "w", encoding="utf-8") as f:
    # nbformat.write(nb, f)